In [1]:
import os, time, h5py
from tqdm import tqdm
import numpy as np
from scipy import ndimage
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

def check_torch():
    torch_version, cuda_avail = torch.__version__, torch.cuda.is_available()
    count, name = torch.cuda.device_count(), torch.cuda.get_device_name()
    device = torch.device('cuda' if cuda_avail else 'cpu')
    print('-'*60)
    print('----------------------- VERSION INFO -----------------------')
    print('Torch version: {} | Torch Built with CUDA? {}'.format(torch_version, cuda_avail))
    print('# Device(s) available: {}, Name(s): {}'.format(count, name))
    print('-'*60)
    return device

device = check_torch()
cmap, interp = 'binary', 'bicubic'

------------------------------------------------------------
----------------------- VERSION INFO -----------------------
Torch version: 2.7.1+cu126 | Torch Built with CUDA? True
# Device(s) available: 1, Name(s): NVIDIA GeForce RTX 3090
------------------------------------------------------------


In [2]:
with h5py.File('data/facies.h5', 'r') as f:
    facies = f['facies'][:]
print('Facies shape: {}'.format(facies.shape))

Facies shape: (318, 256, 256, 128)


In [24]:
facies_planar = np.expand_dims(np.transpose(facies, (0,2,1,3)).reshape(-1, 256, 128).transpose(0,2,1), 1)
print('Planar facies shape: {}'.format(facies_planar.shape))

Planar facies shape: (81408, 1, 128, 256)


In [ ]:
nwells = 5
extent = (15, 255-16)

depth = np.arange(facies_planar.shape[-2])
nz = len(depth)
wx = np.sort(np.random.randint(extent[0], extent[1], size=(nwells,)))
print('Well x-locations: {}'.format(wx))

well_logs = facies_planar[...,wx]
noise_logs = well_logs + np.random.normal(0, 0.2*facies)

Well x-locations: [ 33  49 121 129 184]


IndexError: index 129 is out of bounds for axis 2 with size 128

In [ ]:
rand_idx = np.random.choice(facies_planar.shape[0], facies_planar.shape[0], replace=False)
train_idx, val_idx, test_idx = rand_idx[:500], rand_idx[500:650], rand_idx[650:800]


y_train, y_val, y_test = facies_planar[train_idx], facies_planar[val_idx], facies_planar[test_idx]
print('Y - train: {} | val: {} | test: {}'.format(y_train.shape, y_val.shape, y_test.shape))

Y - train: (500, 1, 128, 256) | val: (150, 1, 128, 256) | test: (150, 1, 128, 256)


***
# END